In [ ]:
import random
from collections import deque
import math
import random

### 1. Data Generation

Below we generate 10000 unique keys between 100000 and 200000, and store them in a list 'records'.

In [2]:
def generate_unique_keys(num_keys=10000, lower_bound=100000, upper_bound=200000):    
    keys = random.sample(range(lower_bound, upper_bound + 1), num_keys)
    return sorted(keys) 

records = generate_unique_keys()
print("Total records generated:", len(records))

Total records generated: 10000


### 2. Building a B+ Tree

In [20]:
from collections import deque

class BPlusTreeNode:
    def __init__(self, is_leaf=False):
        self.keys = []
        self.children = []
        self.is_leaf = is_leaf
        self.next = None  # used to link leaf nodes together


class BPlusTree:
    def __init__(self, order):
        self.root = BPlusTreeNode(is_leaf=True)
        self.order = order

    def insert(self, key):
        root = self.root

        # if root is full, we need to split it first
        if len(root.keys) == self.order:
            new_root = BPlusTreeNode()
            new_root.children.append(root)
            self.split_child(new_root, 0)
            self.root = new_root

        # now insert into the appropriate node
        self.insert_non_full(self.root, key)

    def insert_non_full(self, node, key):
        if node.is_leaf:
            # find the correct position to insert the key
            idx = self.find_pos(node.keys, key)
            node.keys.insert(idx, key)
        else:
            # navigate to the right child
            idx = self.find_pos(node.keys, key)
            child = node.children[idx]

            # if child is full, split it before going deeper
            if len(child.keys) == self.order:
                self.split_child(node, idx)
                # after split, check if key should go to right child
                if key > node.keys[idx]:
                    idx += 1

            self.insert_non_full(node.children[idx], key)

    def split_child(self, parent, idx):
        order = self.order
        node_to_split = parent.children[idx]
        new_node = BPlusTreeNode(is_leaf=node_to_split.is_leaf)

        mid = order // 2

        if node_to_split.is_leaf:
            # split the keys and link the new node
            new_node.keys = node_to_split.keys[mid:]
            node_to_split.keys = node_to_split.keys[:mid]

            new_node.next = node_to_split.next
            node_to_split.next = new_node

            # promote the first key of new_node to parent
            parent.keys.insert(idx, new_node.keys[0])
        else:
            # middle key goes up, rest split between old and new node
            parent.keys.insert(idx, node_to_split.keys[mid])
            new_node.keys = node_to_split.keys[mid + 1:]
            node_to_split.keys = node_to_split.keys[:mid]

            new_node.children = node_to_split.children[mid + 1:]
            node_to_split.children = node_to_split.children[:mid + 1]

        # insert the new child to the parent
        parent.children.insert(idx + 1, new_node)

    def find_pos(self, keys, key):
        for i in range(len(keys)):
            if key < keys[i]:
                return i
        return len(keys)

    def print_tree(self):
        # level-order print of the tree
        q = deque([(self.root, 0)])
        levels = []

        while q:
            node, lvl = q.popleft()
            if len(levels) <= lvl:
                levels.append([])
            levels[lvl].append(node.keys)

            if not node.is_leaf:
                for child in node.children:
                    q.append((child, lvl + 1))

        for i, level in enumerate(levels):
            print(f"Level {i}: {level}")


In [36]:
def build_dense_tree(records, order):
    print(f"\nBuilding Dense B+ Tree of order {order}...")
    tree = BPlusTree(order)
    for key in records:
        tree.insert(key)
    return tree

def build_sparse_tree(records, order):
    print(f"\nBuilding Sparse B+ Tree of order {order}...")
    tree = BPlusTree(order)

    # Insert only every 10th key to make it sparse
    sparse_records = records[::10]
    for key in sparse_records:
        tree.insert(key)
    
    return tree



In [37]:
# Assuming records from Part 1
records = generate_unique_keys()

# Build dense and sparse trees of order 13
dense_tree_13 = build_dense_tree(records, order=13)
sparse_tree_13 = build_sparse_tree(records, order=13)

# Optional: Print the tree structure
print("\nDense Tree (Order 13):")
dense_tree_13.print_tree()

print("\nSparse Tree (Order 13):")
sparse_tree_13.print_tree()



Building Dense B+ Tree of order 13...

Building Sparse B+ Tree of order 13...

Dense Tree (Order 13):
Level 0: [[120633, 141058, 162405]]
Level 1: [[102826, 105892, 108673, 111549, 114589, 117822], [123789, 126785, 129510, 132255, 135263, 138160], [144200, 147063, 150018, 153246, 156460, 159132], [165171, 167828, 170727, 173492, 176265, 179389, 182478, 185365, 188569, 191276, 194208, 197094]]
Level 2: [[100513, 100865, 101232, 101624, 102020, 102334], [103297, 103734, 104184, 104647, 105007, 105462], [106260, 106656, 107052, 107431, 107777, 108254], [109214, 109566, 110029, 110374, 110758, 111168], [111950, 112410, 112897, 113247, 113682, 114224], [115090, 115538, 116151, 116488, 116969, 117382], [118161, 118558, 119080, 119439, 119834, 120195], [121037, 121630, 122025, 122442, 122792, 123261], [124272, 124619, 125031, 125461, 125845, 126219], [127118, 127479, 127873, 128318, 128676, 129021], [129930, 130344, 130763, 131168, 131565, 131902], [132732, 133185, 133594, 134028, 134517, 13

### 3. Operations

In [38]:
def search(self, key):
    node = self.root
    while not node.is_leaf:
        i = self._find_insert_position(node.keys, key)
        node = node.children[i]
    if key in node.keys:
        print(f"Key {key} found in leaf node with keys: {node.keys}")
        return key
    else:
        print(f"Key {key} not found. Leaf node keys: {node.keys}")
        return None

def range_search(self, start, end):
    node = self.root
    # Go to the leftmost leaf that could contain 'start'
    while not node.is_leaf:
        i = self._find_insert_position(node.keys, start)
        node = node.children[i]

    result = []
    while node:
        for key in node.keys:
            if start <= key <= end:
                result.append(key)
            elif key > end:
                return result
        node = node.next
    return result


In [39]:
def delete(self, key):
    node = self.root
    parent_stack = []

    while not node.is_leaf:
        i = self._find_insert_position(node.keys, key)
        parent_stack.append((node, i))
        node = node.children[i]

    if key in node.keys:
        print(f"Deleting key {key} from node with keys: {node.keys}")
        node.keys.remove(key)
    else:
        print(f"Key {key} not found in tree for deletion.")


### 4. Experiments

In [40]:
import random
from copy import deepcopy

# === PART 4: EXPERIMENTS ===


# Simulate deletion (mocked: doesn't modify tree for now)
def mock_delete(tree, key):
    print(f"\n[DELETE] Trying to delete key {key}")
    print("[MOCK] Deletion not implemented. This is a placeholder.")
    # If needed: implement full deletion logic in B+ tree

# Print all nodes affected during insert/search
def trace_insert(tree, key):
    print(f"\n[INSERT] Inserting key {key}")
    path = []
    node = tree.root
    while not node.is_leaf:
        path.append(deepcopy(node.keys))
        idx = tree.find_pos(node.keys, key)
        node = node.children[idx]
    path.append(deepcopy(node.keys))
    print(f"[TRACE] Path to leaf for insertion: {path}")
    tree.insert(key)
    print(f"[RESULT] After insertion:")
    tree.print_tree()

def trace_search(tree, key):
    print(f"\n[SEARCH] Searching for key {key}")
    node = tree.root
    path = []
    while not node.is_leaf:
        path.append(deepcopy(node.keys))
        idx = tree.find_pos(node.keys, key)
        node = node.children[idx]
    path.append(deepcopy(node.keys))
    found = key in node.keys
    print(f"[TRACE] Path to leaf: {path}")
    print(f"[RESULT] Key {'found' if found else 'not found'} in leaf: {node.keys}")

def run_experiments():
    # Step (a) Generate records
    records = generate_unique_keys()

    # Step (b) Build 4 B+ Trees
    orders = [13, 24]
    trees = {}
    for order in orders:
        dense = build_dense_tree(records, order)
        sparse = build_sparse_tree(records, order)
        trees[f"dense_{order}"] = dense
        trees[f"sparse_{order}"] = sparse

    # Step (c1) Apply 2 random inserts to dense trees
    for label in trees:
        if label.startswith("dense"):
            print(f"\n===> Performing 2 inserts on {label}")
            for _ in range(2):
                key = random.randint(10001, 20000)  # Insert new keys
                trace_insert(trees[label], key)

    # Step (c2) Apply 2 random deletions to sparse trees (mocked)
    for label in trees:
        if label.startswith("sparse"):
            print(f"\n===> Performing 2 deletions on {label}")
            for _ in range(2):
                key = random.choice(records)
                mock_delete(trees[label], key)

    # Step (c3) Apply 5 additional random insertions/deletions to all trees
    for label in trees:
        print(f"\n===> Performing 5 mixed operations on {label}")
        for _ in range(5):
            op = random.choice(["insert", "delete"])
            key = random.randint(10001, 20000) if op == "insert" else random.choice(records)
            if op == "insert":
                trace_insert(trees[label], key)
            else:
                mock_delete(trees[label], key)

    # Step (c4) Apply 5 random searches to all trees
    for label in trees:
        print(f"\n===> Performing 5 searches on {label}")
        for _ in range(5):
            key = random.randint(1, 20000)
            trace_search(trees[label], key)

# Run the full experiment
run_experiments()



Building Dense B+ Tree of order 13...

Building Sparse B+ Tree of order 13...

Building Dense B+ Tree of order 24...

Building Sparse B+ Tree of order 24...

===> Performing 2 inserts on dense_13

[INSERT] Inserting key 10038
[TRACE] Path to leaf for insertion: [[120169, 140820, 160838], [103064, 105873, 109088, 111967, 114673, 117451], [100430, 100798, 101256, 101654, 102057, 102588], [100051, 100112, 100186, 100203, 100304, 100378], [100019, 100021, 100035, 100037, 100041, 100042]]
[RESULT] After insertion:
Level 0: [[120169, 140820, 160838]]
Level 1: [[103064, 105873, 109088, 111967, 114673, 117451], [123004, 125763, 128804, 131600, 134749, 137829], [143533, 146522, 149201, 152389, 155231, 157956], [163844, 166783, 169606, 172850, 175846, 178767, 181596, 184787, 187643, 191093, 194200, 197247]]
Level 2: [[100430, 100798, 101256, 101654, 102057, 102588], [103488, 103834, 104258, 104584, 104970, 105510], [106387, 106857, 107353, 107746, 108173, 108617], [109422, 109826, 110286, 11069